In [ ]:
!pip install msal
!pip install azure-identity azure-keyvault-secrets


In [ ]:
#Migrates workspaces to F Capacity based on Capacity and Workspace IDs to move to Capacity


import os
from msal import ConfidentialClientApplication
from azure.identity import EnvironmentCredential
from azure.keyvault.secrets import SecretClient
import requests
import json

# Set environment variables (for development purposes)
os.environ['AZURE_CLIENT_ID'] = '<CLIENT ID>'
os.environ['AZURE_TENANT_ID'] = '<TENANT ID>'
os.environ['AZURE_CLIENT_SECRET'] = '<CLIENT SECRET>'

# Fetch environment variables
tenant_id = os.getenv('AZURE_TENANT_ID')
client_id = os.getenv('AZURE_CLIENT_ID')

if not tenant_id or not client_id:
    raise Exception("Missing required environment variables: AZURE_TENANT_ID, AZURE_CLIENT_ID")

authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["https://api.fabric.microsoft.com/.default"]
key_vault_name = "<KEY_VAULT_NAME>"  # Replace with your Key Vault name
secret_name = "<KEY_VAULT_SECRET>"  # Replace with your secret name

# Get client secret from Azure Key Vault
vault_url = f"https://{key_vault_name}.vault.azure.net"
credential = EnvironmentCredential()

# Debugging: Verify that the credential works
try:
    token = credential.get_token("https://vault.azure.net/.default")
    print("Successfully acquired token for Key Vault.")
except Exception as e:
    print("Failed to acquire token for Key Vault:", str(e))
    raise

secret_client = SecretClient(vault_url=vault_url, credential=credential)

# Debugging: Verify secret retrieval
try:
    retrieved_secret = secret_client.get_secret(secret_name)
    client_secret = retrieved_secret.value
    print("Successfully retrieved secret from Azure Key Vault.")
except Exception as e:
    print("Failed to retrieve secret from Azure Key Vault:", str(e))
    raise

# Debugging: Print client secret to ensure it's not empty or incorrect
if not client_secret:
    raise Exception("Client secret retrieved from Key Vault is empty.")

print("Client secret retrieved:", client_secret)

# Create a confidential client application
app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

# Acquire token (non-interactive)
result = app.acquire_token_for_client(scopes)

if 'access_token' in result:
    access_token = result['access_token']
    headers = {'Authorization': 'Bearer ' + access_token}

    # Get list of capacities
    capacities_url = "https://api.fabric.microsoft.com/v1/capacities"
    capacities_response = requests.get(capacities_url, headers=headers)

    if capacities_response.status_code == 200:
        capacities = capacities_response.json()
        print("Capacities:")
        for capacity in capacities['value']:
            print(f"ID: {capacity['id']}")
            print(f"Name: {capacity['displayName']}")
            print(f"SKU: {capacity['sku']}")
            print(f"Region: {capacity['region']}")
            print(f"State: {capacity['state']}")
            print("--------")
    else:
        print("Failed to get capacities:", capacities_response.status_code, capacities_response.text)
        capacities = {'value': []}

    # Get list of workspaces
    workspaces_url = "https://api.fabric.microsoft.com/v1/workspaces"
    response = requests.get(workspaces_url, headers=headers)

    if response.status_code == 200:
        workspaces = response.json()
        print("Workspaces:")
        workspace_list = []
        for workspace in workspaces['value']:
            workspace_list.append({
                'id': workspace['id'],
                'name': workspace['displayName'],
                'description': workspace['description'],
                'type': workspace['type'],
                'capacityId': workspace.get('capacityId', 'N/A')
            })
            print(f"ID: {workspace['id']}")
            print(f"Name: {workspace['displayName']}")
            print(f"Description: {workspace['description']}")
            print(f"Type: {workspace['type']}")
            print(f"Capacity ID: {workspace.get('capacityId', 'N/A')}")
            print("--------")

        # Select capacity and workspaces for assignment
        if capacities['value']:
            selected_capacity_id = input("Enter Capacity ID to assign workspaces to: ")

            print("Select workspaces to assign (comma-separated IDs):")
            for workspace in workspace_list:
                print(f"ID: {workspace['id']}, Name: {workspace['name']}, Description: {workspace['description']}, Capacity ID: {workspace['capacityId']}")
            selected_workspace_ids = input("Enter Workspace IDs: ").split(",")

            # Assign workspaces to the selected capacity
            for workspace_id in selected_workspace_ids:
                assign_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/assignToCapacity"
                data = {"capacityId": selected_capacity_id}
                assign_response = requests.post(assign_url, headers=headers, json=data)

                if assign_response.status_code == 200 or 202:
                    print(f"Workspace {workspace_id} assigned successfully to capacity {selected_capacity_id}.")
                else:
                    print(f"Failed to assign workspace {workspace_id}: {assign_response.status_code} {assign_response.text}")
        else:
            print("No capacities available to select.")
    else:
        print("Failed to get workspaces:", response.status_code, response.text)
else:
    print("Token acquisition failed:", result.get("error"), result.get("error_description"))

In [ ]:
#Migrates workspaces to F Capacity based on Capacity ID Mapping

import os
from msal import ConfidentialClientApplication
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import requests
import json
import time

# Set environment variables
os.environ['AZURE_CLIENT_ID'] = '<CLIENT ID>'
os.environ['AZURE_TENANT_ID'] = '<TENANT ID>'
os.environ['AZURE_CLIENT_SECRET'] = '<CLIENT SECRET>'

# Fetch environment variables
tenant_id = os.getenv('AZURE_TENANT_ID')
client_id = os.getenv('AZURE_CLIENT_ID')

if not tenant_id or not client_id:
    print("Missing required environment variables: AZURE_TENANT_ID, AZURE_CLIENT_ID")
    raise Exception("Missing required environment variables: AZURE_TENANT_ID, AZURE_CLIENT_ID")

authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["https://api.fabric.microsoft.com/.default"]
key_vault_name = "<KEY_VAULT_NAME>"  # Replace with your Key Vault name
secret_name = "<KEY_VAULT_SECRET>"  # Replace with your secret name

# Get client secret from Azure Key Vault
vault_url = f"https://{key_vault_name}.vault.azure.net"
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=vault_url, credential=credential)

# Retrieve client secret
try:
    retrieved_secret = secret_client.get_secret(secret_name)
    client_secret = retrieved_secret.value
    print("Successfully retrieved secret from Azure Key Vault.")
except Exception as e:
    print(f"Failed to retrieve secret from Azure Key Vault: {str(e)}")
    raise

# Create a confidential client application
app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

# Acquire token (non-interactive)
result = app.acquire_token_for_client(scopes)

if 'access_token' in result:
    access_token = result['access_token']
    headers = {'Authorization': 'Bearer ' + access_token}

    # Get list of capacities
    capacities_url = "https://api.fabric.microsoft.com/v1/capacities"
    capacities_response = requests.get(capacities_url, headers=headers)
    capacities = capacities_response.json()["value"] if capacities_response.status_code == 200 else []
    if capacities:
        print("Successfully retrieved capacities:")
        for capacity in capacities:
            print(f"ID: {capacity['id']}")
            print(f"Name: {capacity['displayName']}")
            print(f"SKU: {capacity['sku']}")
            print(f"Region: {capacity['region']}")
            print(f"State: {capacity['state']}")
            print("--------")
    else:
        print(f"Failed to get capacities: {capacities_response.status_code}, {capacities_response.text}")

    # Get list of workspaces
    workspaces_url = "https://api.fabric.microsoft.com/v1/workspaces"
    workspaces_response = requests.get(workspaces_url, headers=headers)
    workspaces = workspaces_response.json()["value"] if workspaces_response.status_code == 200 else []
    if workspaces:
        print("Successfully retrieved workspaces:")
        for workspace in workspaces:
            print(f"ID: {workspace['id']}")
            print(f"Name: {workspace['displayName']}")
            print(f"Description: {workspace['description']}")
            print(f"Type: {workspace['type']}")
            print(f"Capacity ID: {workspace.get('capacityId', 'N/A')}")
            print("--------")
    else:
        print(f"Failed to get workspaces: {workspaces_response.status_code}, {workspaces_response.text}")

    # Define the mapping from legacy capacities to new Fabric capacities using capacity ID
    capacity_mapping = {
        "2e931b9d-dbbf-41b9-5555-3333333333": "e1be100d-d431-47cf-8889-4444444444"
        # Add more mappings as needed
    }

    # Prepare and display the mappings
    mappings = []
    for workspace in workspaces:
        workspace_name = workspace.get("displayName")
        legacy_capacity_id = workspace.get("capacityId")
        new_capacity_id = capacity_mapping.get(legacy_capacity_id)

        if new_capacity_id:
            mappings.append((workspace_name, workspace['id'], legacy_capacity_id, new_capacity_id))

    # Display the mappings
    if mappings:
        print("Workspaces and their mapped capacities:")
        for mapping in mappings:
            print(f"Workspace: {mapping[0]} (ID: {mapping[1]})")
            print(f"  Legacy Capacity ID: {mapping[2]}")
            print(f"  New Capacity ID: {mapping[3]}")
            print("  --------")
    else:
        print("No valid mappings found.")

    # Confirm before proceeding with the migration
    confirm = input("Do you want to proceed with the migration? (yes/no): ").strip().lower()
    if confirm != 'yes':
        print("Migration aborted by user.")
        exit()

    # Migrate workspaces based on the confirmed mappings
    for mapping in mappings:
        workspace_name, workspace_id, legacy_capacity_id, new_capacity_id = mapping
        assign_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/assignToCapacity"
        data = {"capacityId": new_capacity_id}
        assign_response = requests.post(assign_url, headers=headers, json=data)

        if assign_response.status_code == 200:
            print(f"Workspace {workspace_name} (ID: {workspace_id}) assigned successfully to new capacity (ID: {new_capacity_id}).")
        else:
            print(f"Failed to assign workspace {workspace_name} (ID: {workspace_id}): {assign_response.status_code} {assign_response.text}")

        # Optional: Delay between migrations to manage load
        time.sleep(1)
else:
    print(f"Token acquisition failed: {result.get('error')}, {result.get('error_description')}")
